In [1]:
import pandas as pd

In [3]:

df = pd.read_csv('SMSSpamCollection', sep='\t',
                           names=["label", "message"])


In [4]:
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [5]:
df.shape

(5572, 2)

In [6]:
df["label"].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [7]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aniket/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm_notebook as tqdm

In [9]:
ps = PorterStemmer()

In [10]:



corpus = []  

with tqdm(total=len(df)) as pbar:
    for i in range(0, len(df)):
        review = re.sub('[^a-zA-Z]', ' ', df['message'][i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)
        pbar.update(1)

/var/folders/38/w2xvkyxd2k37kxqm6g11c6fw0000gn/T/ipykernel_2399/2597300154.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm(total=len(df)) as pbar:


  0%|          | 0/5572 [00:00<?, ?it/s]

In [11]:
corpus[:10]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free']

### Using BOW MODEL

In [12]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [14]:
y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [15]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [16]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)


In [17]:
y_pred=spam_detect_model.predict(X_test)


In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [19]:
cm = confusion_matrix(y_test,y_pred)

In [20]:
cm

array([[946,   9],
       [  7, 153]])

In [21]:
acc = accuracy_score(y_test, y_pred)

In [22]:
acc

0.9856502242152466

### TF-IDF 

In [23]:
# Creating the TF-IDF model
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer()
X_tf = cv.fit_transform(corpus).toarray()


In [24]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size = 0.20, random_state = 0)


In [25]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)


In [26]:
y_pred=spam_detect_model.predict(X_test)


In [27]:
cm = confusion_matrix(y_test,y_pred)

In [28]:
cm

array([[955,   0],
       [ 34, 126]])

In [29]:
acc = accuracy_score(y_test, y_pred)
acc

0.9695067264573991